In [1]:
import random
import os
import numpy as np
import pandas as pd
from datetime import datetime
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)

# ==========================================
# 1. 데이터 로드
# ==========================================

train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
target = '임신 성공 여부'

# ==========================================
# 2. feature_engineering
# ==========================================

def feature_engineering(df):
    
    # ---------------------------------------------------------
    # 1. 수치형 변환 및 기초 클리닝 (derive_features 로직)
    # ---------------------------------------------------------
    count_cols = [
        '총 시술 횟수', '클리닉 내 총 시술 횟수', '총 임신 횟수', '총 출산 횟수',
        'IVF 시술 횟수', 'DI 시술 횟수', 'IVF 임신 횟수', 'DI 임신 횟수',
        'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '이식된 배아 수', 
        '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '저장된 배아 수'
    ]
    
    for col in count_cols:
        if col in df.columns and df[col].dtype == 'object':
            # '회', ' 이상' 등을 제거하고 공백 정리 후 숫자형으로 변환
            df[col] = df[col].astype(str).str.replace(r'회| 이상', '', regex=True).str.strip()
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    # DI 시술 유형에 대한 결측치 처리 (특정 조건에 따른 0 채움)
    target_cols = [
            '총 생성 배아 수', '기증자 정자와 혼합된 난자 수', '파트너 정자와 혼합된 난자 수',
            '미세주입된 난자 수', '혼합된 난자 수', '저장된 신선 난자 수', '해동 난자 수',
            '해동된 배아 수', '미세주입 후 저장된 배아 수', '저장된 배아 수',
            '미세주입 배아 이식 수', '이식된 배아 수', '미세주입에서 생성된 배아 수',
            '수집된 신선 난자 수', '단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '대리모 여부',
            '기증 배아 사용 여부', '신선 배아 사용 여부', '동결 배아 사용 여부'
        ]
    
    # 해당 컬럼들이 존재하는 경우에만 처리
    existing_target_cols = [c for c in target_cols if c in df.columns]
    if '특정 시술 유형' in df.columns:
        mask_di = df['특정 시술 유형'] == 'DI'
        df.loc[mask_di, existing_target_cols] = df.loc[mask_di, existing_target_cols].fillna(0)

    # ---------------------------------------------------------
    # 2. 문자열 기반 파생 변수 생성 (수치 변환 전 수행) (preprocess 로직)
    # ---------------------------------------------------------
    
    # 2.1 시술_대분류 & BLASTOCYST
    if '특정 시술 유형' in df.columns:
        def major_procedure(x):
            if pd.isna(x): return "Unknown"
            if "IUI" in x: return "IUI"
            if "DI" in x: return "Other"
            if "ICSI" in x: return "ICSI"
            if "IVF" in x: return "IVF"
            return "Other"
        
        df["시술_대분류"] = df["특정 시술 유형"].apply(major_procedure)
        df["BLASTOCYST_포함"] = df["특정 시술 유형"].str.contains("BLASTOCYST", na=False).astype(int)

    # 2.2 총시술_bin3 (위에서 수치형으로 변환된 '총 시술 횟수' 활용)
    def collapse_trials(x):
        # 이미 수치형으로 변환되었으므로 숫자 기준으로 처리하거나 문자열 매핑 유지
        # 원본 로직이 문자열('0회')을 처리했으나, 앞서 수치형 변환을 했으므로 숫자로 처리
        if x == 0: return '0회'
        elif 1 <= x <= 2: return '1–2회'
        else: return '3회 이상'
        
    df["총시술_bin3"] = df["총 시술 횟수"].apply(collapse_trials)

    # 2.3 나이_3구간 (원본 문자열 '만xx-xx세' 활용)
    def age_group_simple(age):
        if age == '알 수 없음': return 'Unknown'
        elif age == '만18-34세': return '34세 이하'
        elif age in ['만35-37세', '만38-39세']: return '35-39세'
        else: return '40세 이상'

    df['나이_3구간'] = df['시술 당시 나이'].apply(age_group_simple)

    # ---------------------------------------------------------
    # 3. 불임 원인 및 복잡도 계산 (Column Drop 전 수행)
    # ---------------------------------------------------------
    female_cols = [
        '불임 원인 - 여성 요인', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애', 
        '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '여성 주 불임 원인', 
        '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인'
    ]
    male_cols = [
        '불임 원인 - 남성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', 
        '불임 원인 - 정자 형태', '불임 원인 - 정자 면역학적 요인', '남성 주 불임 원인', 
        '남성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인'
    ]
    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]

    # 불임 원인 복잡도 (preprocess)
    df["불임_원인_개수"] = df[infertility_cols].sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0: return 'None'
        elif count == 1: return 'Single'
        elif count == 2: return 'Double'
        else: return 'Multiple'
    
    df['불임원인_복잡도'] = df['불임_원인_개수'].apply(infertility_complexity)

    # ---------------------------------------------------------
    # 4. 배아 및 난자 관련 효율/상태 변수 생성
    # ---------------------------------------------------------
    epsilon = 1e-6
    
    # 4.1 난자 총합 및 손실률 (derive_features)
    oocyte_cols = ['수집된 신선 난자 수', '혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '해동 난자 수']
    df['총 난자 수'] = df[oocyte_cols].fillna(0).sum(axis=1)
    df['배아 손실률'] = (df['총 난자 수'] - df['총 생성 배아 수']) / (df['총 난자 수'] + epsilon)

    # 4.2 배아 이식 상태 및 단계 (preprocess)
    embryo_stage_cols = [
        "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "배아 생성 주요 이유",
        "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수",
        "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수",
        "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부",
    ]
    df["배아_이식_미도달"] = df[embryo_stage_cols].isna().all(axis=1).astype(int)
    df["배아_이식_여부"] = 1 - df["배아_이식_미도달"]

    def embryo_stage(row):
        if row['배아_이식_여부'] == 0: return '배아단계_미도달'
        elif pd.isna(row['총 생성 배아 수']) or row['총 생성 배아 수'] == 0: return '배아생성_실패'
        elif pd.isna(row['이식된 배아 수']) or row['이식된 배아 수'] == 0: return '이식_미실시'
        else: return '이식_완료'

    df['배아_진행_단계'] = df.apply(embryo_stage, axis=1)

    # 4.3 이식 배아 구간 (preprocess)
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0: return '0개'
        elif count <= 2: return '1-2개'
        else: return '3개 이상'
    df['이식배아_구간'] = df['이식된 배아 수'].apply(embryo_count_bin)

    # 4.4 기타 상태 플래그
    df['Day5_이식_여부'] = (df['배아 이식 경과일'] == 5.0).astype(int)

    # ---------------------------------------------------------
    # 5. 수치형 변환(Mapping) 및 비율 계산 (derive_features 메인 로직)
    # ---------------------------------------------------------
    
    # 5.1 나이 Mapping (Ordinal Encoding)
    # *주의: 이 단계 이후로 '시술 당시 나이'는 숫자가 됨
    age_order = {
        '만18-34세': 1, '만35-37세': 2, '만38-39세': 3, 
        '만40-42세': 4, '알 수 없음': 5, '만43-44세': 6, '만45-50세': 7,
    }
    df['시술 당시 나이'] = df['시술 당시 나이'].map(age_order).fillna(5)

    # 5.2 비율 및 효율성 변수
    df['나이 이식배아 비율'] = df['시술 당시 나이'] * df['이식된 배아 수']
    df['연령별 배아 효율'] = df['이식된 배아 수'] / df['시술 당시 나이']
    df['배아_생성_효율'] = df['총 생성 배아 수'] / (df['수집된 신선 난자 수'] + 1)
    df['배아_이식_비율'] = df['이식된 배아 수'] / (df['총 생성 배아 수'] + 1)
    df['배아_저장_비율'] = df['저장된 배아 수'] / (df['총 생성 배아 수'] + 1)

    # ---------------------------------------------------------
    # 6. 범주형 데이터 단순화 (Categorical Cleaning)
    # ---------------------------------------------------------
    
    # 6.1 시술 유형 정제
    def clean_treatment(val):
        val = str(val).upper()
        if 'ICSI' in val: return 'ICSI'
        if 'IVF' in val: return 'IVF'
        if 'IUI' in val: return 'IUI'
        return 'Other'
    df['특정 시술 유형'] = df['특정 시술 유형'].apply(clean_treatment)

    # 6.2 배아 생성 이유 정제
    def clean_reason(x):
        if pd.isna(x): return 'Unknown'
        x = str(x)
        if '시술용' in x: return 'Treatment'
        if '기증' in x: return 'Donation'
        if '저장' in x: return 'Storage'
        return 'Other'
    df['배아 생성 주요 이유'] = df['배아 생성 주요 이유'].apply(clean_reason)

    # ---------------------------------------------------------
    # 7. 교호작용(Interaction) 변수 생성 (preprocess)
    # ---------------------------------------------------------
    # '시술 당시 나이'가 이미 숫자로 변환되었으므로 astype(str) 사용 시 "1.0_0" 형태로 생성됨.
    # 이는 One-hot encoding 등에 유리하므로 그대로 진행
    df['나이×Day5'] = df['시술 당시 나이'].astype(str) + '_' + df['Day5_이식_여부'].astype(str)
    df['시술횟수×나이'] = df['총시술_bin3'] + '_' + df['나이_3구간']

    # ---------------------------------------------------------
    # 8. 불필요 컬럼 삭제 (derive_features)
    # ---------------------------------------------------------
    drop_cols = [
        'ID', 
        '시술 시기 코드',
        '저장된 배아 수', 
        '착상 전 유전 검사 사용 여부', 
        'PGS 시술 여부', 
        'DI 출산 횟수', 
        '대리모 여부', 
        '시술 유형',
        '저장된 신선 난자 수',
        '총 난자 수',
        '클리닉 내 총 시술 횟수'
    ]
    # 사용된 원본 컬럼들 모두 삭제
    final_drop_list = drop_cols + female_cols + male_cols
    df.drop(columns=final_drop_list, errors='ignore', inplace=True)
    
    return df

train_df = feature_engineering(train_df)
test_df = feature_engineering(test_df)

# ==========================================
# 3. 데이터 타입 최적화
# ==========================================

def optimize_memory(df):
    for col in df.columns:
        # 숫자형 데이터 최적화
        if df[col].dtype in ['int64', 'float64']:
            if df[col].dtype == 'int64':
                df[col] = pd.to_numeric(df[col], downcast='integer')
            else:
                df[col] = pd.to_numeric(df[col], downcast='float')
        
        # 문자열(object) 데이터 -> 범주형(category) 변환
        # AutoGluon은 category 타입을 매우 효율적으로 처리합니다.
        elif df[col].dtype == 'object':
            df[col] = df[col].astype('category')
    return df

train_df = optimize_memory(train_df)
test_df = optimize_memory(test_df)

# ==========================================
# 4. 모델 학습 설정
# ==========================================

predictor = TabularPredictor(
    label=target,
    eval_metric='roc_auc',
    problem_type='binary',
    path='ag_models_out_v15',
).fit(
    train_data=train_df,
    time_limit=10800,
    presets='best_quality',
    num_stack_levels=1,
    num_bag_folds=8,
    num_bag_sets=2,
    included_model_types=['GBM', 'CAT', 'XGB'],
    refit_full=True,
    dynamic_stacking=False,
    save_space=True,
    set_best_to_refit_full=True
)

# ==========================================
# 4. 예측 (Test Data 활용) - 최종 결과를 확률로 출력 (Positive 클래스에 대한 확률만 추출)
# ==========================================

pred_probs = predictor.predict_proba(test_df)
final_probs = pred_probs.iloc[:, 1]

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:40 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       8.14 GB / 16.00 GB (50.9%)
Disk Space Avail:   163.26 GB / 460.43 GB (35.5%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=2
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "/Users/admin/00 임신 예측/pregnancy-outcome-prediction-ai/Notebooks/ag_models_out_v15"
Train Data Rows:    256351
Train Data Columns:

In [2]:
# --- 리더보드 출력 ---
lb = predictor.leaderboard(train_df, silent=True)
display(lb.head())

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_r161_BAG_L1_FULL,0.783320,NaN,roc_auc,2.060187,NaN,52.960176,2.060187,NaN,52.960176,1,True,66
1,LightGBM_r161_BAG_L1,0.783307,0.737108,roc_auc,34.745983,45.693217,65.377847,34.745983,45.693217,65.377847,1,True,19
2,XGBoost_r33_BAG_L1_FULL,0.779560,NaN,roc_auc,1.194533,NaN,4.513463,1.194533,NaN,4.513463,1,True,57
3,XGBoost_r33_BAG_L1,0.778641,0.736634,roc_auc,19.760887,11.014163,80.926188,19.760887,11.014163,80.926188,1,True,10
4,LightGBMLarge_BAG_L1,0.766988,0.737602,roc_auc,9.486379,8.882177,20.180420,9.486379,8.882177,20.180420,1,True,5


In [3]:
# --- 제출 파일 생성 ---
submission = pd.read_csv('../Data/sample_submission.csv')
submission['probability'] = final_probs.values

# 현재 시간 가져오기 (예: 0206_1031)
now = datetime.now().strftime('%m%d_%H%M')
file_name = f"{now}_submission.csv"
submission.to_csv(file_name, index=False)

print(f"학습 및 예측이 완료되었습니다. 결과가 {file_name}에 저장되었습니다.")

학습 및 예측이 완료되었습니다. 결과가 0210_1500_submission.csv에 저장되었습니다.


In [4]:
# --- 피처 중요도 ---
fi = predictor.feature_importance(data=train_df.sample(n=min(5000, len(train_df)), random_state=42))
fi.to_excel('fi15.xlsx')

These features in provided data are not utilized by the predictor and will be ignored: ['시술_대분류']
Computing feature importance via permutation shuffling for 60 features using 5000 rows with 5 shuffle sets...
	197.39s	= Expected runtime (39.48s per shuffle set)
	127.99s	= Actual runtime (Completed 5 of 5 shuffle sets)
